In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np
import keras

In [ ]:
data=pd.read_fwf('/content/drive/My Drive/dataset/spa.txt',header=None,names=['language'])

In [ ]:
data

,language
0,Go.\tVe.
1,Go.\tVete.
2,Go.\tVaya.
3,Go.\tVáyase.
4,Hi.\tHola.
...,...
118959,There are four main causes
118960,There are mothers and fathe
118961,A carbon footprint is the a
118962,Since there are usually mul


In [ ]:
x=[]
y=[]

for i in data['language']:
  i=i.split('\t')
  if i[0]:
    if i[1]:
      x.append(i[0].strip())
      y.append(i[1].strip())

In [ ]:
import re
def preprocess(w):
  w=re.sub(r"([?.!,%&^%$#@])",r" ",w)
  w=re.sub(r'[" "]+'," ",w)
  w=re.sub(r"[^a-zA-Z?.!,]+"," ",w)
  w=w.strip()
  w='<start>'+w+'<end>'
  return w

In [ ]:
train_data=list(map(preprocess,x))

In [ ]:
test_data

In [ ]:
test_data=list(map(preprocess,y))

In [ ]:
train_data1=list(map(preprocess,y))

In [ ]:
train_data1

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [ ]:
max_len=256
max_words=10000


In [ ]:
tokenizer=Tokenizer(num_words=10000)
tokenizer.fit_on_texts(train_data1)
sequences1=tokenizer.texts_to_sequences(train_data1)
word_index=tokenizer.word_index

In [ ]:
tokenizer=Tokenizer(num_words=10000)
tokenizer.fit_on_texts(train_data)
sequences=tokenizer.texts_to_sequences(train_data)
word_index=tokenizer.word_index

In [ ]:
tokenizer=Tokenizer(num_words=10000)
tokenizer.fit_on_texts(test_data)
sequences3=tokenizer.texts_to_sequences(test_data)
word_index=tokenizer.word_index

In [ ]:
enc_data=pad_sequences(sequences,maxlen=max_len,padding='post')
dec_data=pad_sequences(sequences1,maxlen=max_len,padding='post')
dec_data1=pad_sequences(sequences3,maxlen=max_len,padding='post')

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
encoder_input = keras.layers.Input(shape=[None], dtype=tf.int32)
encoder_embedding = keras.layers.Embedding(input_dim=10001,output_dim=256)(encoder_input)
_, encoder_state_h, encoder_state_c = keras.layers.LSTM(
    128, return_state=True)(encoder_embedding)
encoder_state = [encoder_state_h, encoder_state_c]

decoder_input = keras.layers.Input(shape=[None], dtype=tf.int32)
decoder_embedding = keras.layers.Embedding(
    input_dim=10002,
    output_dim=256)(decoder_input)
decoder_lstm_output = keras.layers.LSTM(128, return_sequences=True)(
    decoder_embedding, initial_state=encoder_state)
decoder_output = keras.layers.Dense(10000,
                                    activation="softmax")(decoder_lstm_output)

In [ ]:
model = keras.models.Model(inputs=[encoder_input, decoder_input],
                           outputs=[decoder_output])

In [ ]:
optimizer = keras.optimizers.Nadam()
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])

In [ ]:
history = model.fit([enc_data,dec_data],dec_data1, epochs=10)